In [21]:
import pycurl
import json
from io import BytesIO
import pandas as pd
import re

In [ ]:
def get_metadata(url, filter):
    """
    Get metadata from the opendata API.

    Parameters:
    url (str): The URL to the API.
    filter (str): The filter to search for.

    Returns:
    dict: A JSON object containing the metadata if the request is successful.
    None: If the request is not successful or the response is empty.
    """
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, f'{url}package_search?q=title:{filter}')
    c.setopt(c.WRITEDATA, buffer)

    c.perform()
    c.close()

    response = buffer.getvalue().decode('utf-8')

    if not response.strip():
        print("The response is empty.")
    else:
        try:
            data = json.loads(response)

            if data.get('success'):
                return data
            else:
                print("API did not return a successful response:", data)
        except json.JSONDecodeError as e:
            print("Failed to decode JSON:", e)
            print("Response content:", response)

def save_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

def filter_result_by_year(data, year):
    for result in data['result']['results']:
        if str(year) in result['name']:
            return result
        
def get_resource_url_for_year(data, year):
    combined_data = {
        'data': [],
        'metadata': []
    }

    result = filter_result_by_year(data, year)

    for resource in result['resources']:
        url = resource['download_url']
        if "Haltepunkt" in url or "Haltestelle" in url:
            combined_data['metadata'].append(url)
        else:
            combined_data['data'].append(url)

    return combined_data

def pack_all_urls_into_one_dict(url, filter):
    data = get_metadata(url, filter)

    all_urls = {}

    for dataset in data['result']['results']:
        year = re.search(r'\d{4}', dataset['name']).group(0)

        all_urls[year] = get_resource_url_for_year(data, year)
    
    all_urls_sorted = dict(sorted(all_urls.items()))

    return all_urls_sorted


In [ ]:
url = "https://ckan.opendata.swiss/api/3/action/"
filter = "Fahrzeiten"
storage_options = {'User-Agent': 'Mozilla/5.0'}


download_url = pack_all_urls_into_one_dict(url, filter)

In [24]:
download_url.keys()

dict_keys(['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'])

In [25]:
download_url['2019']['metadata'][0]

'https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2019/download/Haltestelle.csv'

In [30]:
df = pd.read_csv(download_url['2019']['metadata'][0], storage_options=storage_options)

In [27]:
url = download_url['2018']['metadata'][0]

In [29]:
df

,halt_id,halt_diva,halt_kurz,halt_lang,halt_ist_aktiv
0,143,2570,BirmSte,"Birmensdorf ZH, Sternen/WSL",True
1,309,3356,WaldBir,"Waldegg, Birmensdorferstrasse",True
2,373,6232,FRAF07,"Zürich Flughafen, Fracht",True
3,539,2655,TBAH01,"Thalwil, Bahnhof",True
4,588,3027,FLUG07,"Zürich Flughafen, Bahnhof",True
...,...,...,...,...,...
725,3198,6999,TWÄR,"Binz bei Maur, Twäracher",True
726,3206,7000,SMEU,"Schlieren, Meuchwis",True
727,3217,7022,BAWO,"Zürich, Bahnhof Wollishofen",True
728,3222,7021,SRAI,"Spreitenbach, Raiacker",True


In [11]:
for year in download_url:
    print(year)
    for url in download_url[year]['metadata']:
        print(url)
    

2015
2016
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2016/download/Haltepunkt.csv
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2016/download/Haltestelle.csv
2017
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2017/download/Haltepunkt.csv
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2017/download/Haltestelle.csv
2018
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2018/download/Haltestelle.csv
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2018/download/Haltepunkt.csv
2019
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2019/download/Haltestelle.csv
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2019/download/Haltepunkt.csv
2020
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2020/download/Haltestelle.csv
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2020/download/Haltepunkt.csv
2021
https://data.stadt-zuerich.ch/dataset/vbz_fahrzeiten_ogd_2021/download/Haltepunkt.csv
https://data.s